In [3]:
import warnings
warnings.filterwarnings('ignore')
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.feature import StringIndexer
from pyspark.sql import functions as F

In [2]:
spark = SparkSession.builder.appName('HnMRecommender').getOrCreate()

23/11/02 09:41:21 WARN Utils: Your hostname, Ubuntu resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/11/02 09:41:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/02 09:41:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/11/02 09:41:24 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
customers = spark.read.csv("./customers.csv", inferSchema=True, header=True).cache()

In [5]:
indexer = StringIndexer(inputCol="customer_id", outputCol="user_index")
indexed_user_df = indexer.fit(customers).transform(customers)

# indexer = StringIndexer(inputCol="article_id", outputCol="item_index")
# indexed_item_df = indexer.fit(indexed_user_df).transform(indexed_user_df)

23/11/02 09:42:46 ERROR Executor: Exception in task 1.0 in stage 3.0 (TID 10) 4]
java.lang.OutOfMemoryError: Java heap space
	at org.apache.spark.sql.catalyst.expressions.codegen.BufferHolder.grow(BufferHolder.java:80)
	at org.apache.spark.sql.catalyst.expressions.codegen.UnsafeWriter.grow(UnsafeWriter.java:63)
	at org.apache.spark.sql.catalyst.expressions.codegen.UnsafeWriter.writeUnalignedBytes(UnsafeWriter.java:127)
	at org.apache.spark.sql.catalyst.expressions.codegen.UnsafeWriter.write(UnsafeWriter.java:118)
	at org.apache.spark.sql.catalyst.expressions.codegen.UnsafeWriter.write(UnsafeWriter.java:114)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$SpecificUnsafeProjection.apply(Unknown Source)
	at org.apache.spark.sql.execution.aggregate.ComplexTypedAggregateExpression.serialize(TypedAggregateExpression.scala:267)
	at org.apache.spark.sql.catalyst.expressions.aggregate.TypedImperativeAggregate.serializeAggregateBufferInPlace(interfaces.scala:621)
	at org.apache.spar

Py4JError: An error occurred while calling o32.fit

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/laanith/.local/lib/python3.10/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/laanith/.local/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/laanith/.local/lib/python3.10/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


In [ ]:
print(f'Customers shape : ( {customers.count()}, { len(customers.columns)})')

In [ ]:
transactions = spark.read.csv('./transactions_train.csv', inferSchema=True, header=True)

In [ ]:
print(f'Transactions shape : ( {transactions.count()}, { len(transactions.columns)})')

In [ ]:
transactions.show(5)

In [ ]:
customers.show(5)

In [ ]:
num_unique_customers = transactions.select("customer_id").distinct().count()
print(f'The number of customers that did the transactions : {num_unique_customers}')

In [ ]:
purchases_by_customers = transactions.groupBy("customer_id").agg(F.count("article_id").alias("purchases"))

purchases_by_customers.show(9)

In [ ]:
print(f'The number of customers that did the transactions : {purchases_by_customers.count()}')

In [ ]:
customers_and_transactions = purchases_by_customers.join(customers, "customer_id")
print(f'The resultant unique customers after joining : {customers_and_transactions.count()}')


In [ ]:
customers_and_transactions.show(7)

In [ ]:
articles = spark.read.csv("./articles.csv", inferSchema=True, header=True)
print(f'Articles shape : ( {articles.count()}, { len(articles.columns)})')

In [ ]:
users_per_article = transactions.groupBy("article_id").agg(F.count("customer_id").alias("user_purchases"))
users_per_article.show()

In [ ]:
print(f'The unique articles purchased by customers : {users_per_article.count()}')

In [ ]:
articles_and_transactions = users_per_article.join(articles, "article_id")
print(f'The number of unique articles purchased by customers after joining : {articles_and_transactions.count()}')

In [ ]:
customers_and_transactions.repartition(1).write.csv("./articles_per_user.csv", header='true')
articles_and_transactions.repartition(1).write.csv("./users_per_article.csv", header='true')

In [ ]:
article_per_type_per_user = transactions.groupBy(['customer_id','article_id']).agg(F.count('price').alias('count'))

In [ ]:
article_per_type_per_user.show(5)

In [ ]:
article_per_type_per_user.count()

In [ ]:
article_per_type_per_user.agg(F.max('count')).show()

In [ ]:
article_per_type_per_user.repartition(1).write.csv('output', header='true')

In [ ]:
customers_and_types = customers.join(article_per_type_per_user, "customer_id")

print(f'The total rows in customers_and_types table : { customers_and_types.count()}')

In [ ]:
everything = customers_and_types.join(articles, "article_id")

print(f'The total rows in table of everything : { everything.count()}')

In [ ]:
del article_per_type_per_user

In [ ]:
import gc
gc.collect()

In [ ]:
del articles_and_transactions,customers_and_transactions,customers,num_unique_customers,purchases_by_customers,transactions

In [ ]:
everything.repartition(1).write.csv("outputs", header="true")